# **E T L **

### Version 1.0

---

In [1]:
import pandas as pd
import numpy as np
import os
from converter import DataConverter
from tqdm import tqdm
from sqlalchemy import create_engine

In [2]:
converter = DataConverter() # does conversion etc.
engine = create_engine('sqlite:///..//data/cleaned/DataFrames.db') # do not push to github repository as this will have a size > 100 MB!
data_dir = '../data/raw' # csv files
for file in tqdm(os.listdir(data_dir)): # tqdm -> progressbar
    if file.endswith(".csv"):
        csv_file = os.path.join(data_dir, file)
        symbol = csv_file[12:-4] # get the symbol of '../data/raw\{}.csv', where {} is the symbol
        
        try:
            # get a dataframe from the csv file and convert it into the target shape
            df = pd.read_csv(csv_file)
            df = converter.convert_df(df)
            df = converter.fill_targets(df)

            # scales the values and prepares them for machine learning
            # X and y are numpy arrays which get saved as npz files
            X, y = converter.convert_ml_format(df, symbol)

            # save the dataframe as a table in the DataFrames.db
            df.to_sql(symbol, engine, index=False, if_exists='replace')

            # save the arrays
            np.savez('../data/ml_format/{}.npz'.format(symbol), X=X, y=y)
        except:
            continue

100%|████████████████████████████████████████████████████████████████████████████████| 506/506 [02:42<00:00,  4.00it/s]


In [3]:
#with np.load('../data/ml_format/A.npz') as data:
#    X = data['X']
#    y = data['y']